In [ ]:
# 2) Dataset scan + build data.yaml for pose (robust scan)
from pathlib import Path
import os, yaml
from glob import glob
from collections import Counter

root = Path('./dataset')
assert root.exists(), "Dataset folder './dataset' not found. Please scp/rsync it first."

# Find YOLO label .txt anywhere under labels/ (supports both layouts)
label_txts = []
label_txts += glob(str(root / 'labels' / '**' / '*.txt'), recursive=True)          # type-first: dataset/labels/train/*.txt
label_txts += glob(str(root / '*' / 'labels' / '**' / '*.txt'), recursive=True)    # split-first: dataset/train/labels/*.txt
label_txts = sorted(set(label_txts))
print(f"Found {len(label_txts)} label files under 'labels' folders")

if len(label_txts) == 0:
    # Last resort: search any .txt and warn
    label_txts = glob(str(root / '**' / '*.txt'), recursive=True)
    print(f"Fallback: found {len(label_txts)} .txt files total")
    assert len(label_txts)>0, "No label .txt files found anywhere in './dataset'. Expected YOLO-pose labels under .../labels/..."

# Robustly infer keypoints K by majority across many labels
kp_counter = Counter()
for i, fp in enumerate(label_txts[:2000]):  # sample up to 2000 files
    try:
        with open(fp, 'r') as f:
            for j, ln in enumerate(f):
                ln = ln.strip()
                if not ln:
                    continue
                parts = ln.split()
                # prefer full pose labels: cls cx cy w h + 3*K
                if len(parts) >= 8 and (len(parts) - 5) % 3 == 0:
                    Kcand = (len(parts) - 5) // 3
                    kp_counter[Kcand] += 1
                # stop early per file after a couple of lines
                if j >= 2:
                    break
    except Exception:
        continue

if not kp_counter:
    # Fallback: detect K from kpt-only (cls x1 y1 x2 y2 ...)
    kpt_only_counter = Counter()
    for i, fp in enumerate(label_txts[:2000]):
        try:
            with open(fp, 'r') as f:
                for j, ln in enumerate(f):
                    ln = ln.strip()
                    if not ln:
                        continue
                    parts = ln.split()
                    if len(parts) >= 3 and (len(parts) - 1) % 2 == 0:
                        Kcand = (len(parts) - 1) // 2
                        kpt_only_counter[Kcand] += 1
                    if j >= 2:
                        break
        except Exception:
            continue
    assert kpt_only_counter, 'Could not infer keypoints K from labels.'
    K = max(kpt_only_counter, key=kpt_only_counter.get)
else:
    K = max(kp_counter, key=kp_counter.get)

print(f"Inferred keypoints K (majority) = {K}")

# Heuristics to resolve images dirs for each split
def choose_images_dir(split: str):
    candidates = [
        root / 'images' / split,       # type-first
        root / split / 'images',       # split-first
    ]
    # derive from labels/<split>
    split_label_dirs = []
    for p in label_txts:
        p_norm = p.replace('\\', '/').lower()
        if f"/labels/{split}/" in p_norm:
            split_label_dirs.append(Path(p).parent)
    if split_label_dirs:
        ldir = split_label_dirs[0]
        candidates += [
            ldir.parent / 'images',                  # .../train/images
            ldir.parent.parent / 'images' / ldir.name # .../images/train
        ]
    for c in candidates:
        if c.exists():
            return str(c)
    return None

train_path = choose_images_dir('train')
val_path = choose_images_dir('val')
test_path = choose_images_dir('test')
print('Images dirs (raw):', {'train': train_path, 'val': val_path, 'test': test_path})

# Normalize to be RELATIVE to dataset root if possible, else absolute
root_abs = root.resolve()

def to_rel_or_abs(p):
    if not p:
        return None
    p_abs = Path(p)
    try:
        p_abs = p_abs.resolve()
    except Exception:
        p_abs = (root / p).resolve()
    try:
        return str(p_abs.relative_to(root_abs))
    except ValueError:
        return str(p_abs)

train_rel = to_rel_or_abs(train_path)
val_rel = to_rel_or_abs(val_path)
test_rel = to_rel_or_abs(test_path)
print('Images dirs (normalized):', {'train': train_rel, 'val': val_rel, 'test': test_rel})

assert train_rel and val_rel, (
    "Could not locate images/train and/or images/val. Ensure YOLO structure like:\n"
    "- dataset/images/train & dataset/labels/train\n"
    "- dataset/train/images & dataset/train/labels\n"
)

# Build YAML for pose
names = ['plate']
skeleton = [[0,1],[1,2],[2,3],[3,0]] if K==4 else []
flip_idx = list(range(K))

data = {
    'path': str(root_abs),
    'train': train_rel,
    'val': val_rel,
    'test': test_rel,
    'names': names,
    'kpt_shape': [K, 3],
    'skeleton': skeleton,
    'flip_idx': flip_idx,
}

yaml_path = root/'data.yaml'
with open(yaml_path, 'w') as f:
    yaml.safe_dump(data, f, sort_keys=False)

print('Wrote', yaml_path)
print(yaml.safe_dump(data, sort_keys=False))

# Clear label caches so Ultralytics rebuilds with the corrected kpt_shape
for cache_name in ['train.cache', 'val.cache']:
    cp = root / 'labels' / cache_name
    if cp.exists():
        try:
            os.remove(cp)
            print('Removed cache:', cp)
        except Exception as e:
            print('Failed to remove cache', cp, e)


In [ ]:
# 2.b) Merge extra datasets (datasets2 + dataset3/LP_detection) into ./dataset
from pathlib import Path
import shutil, os, glob

root_main = Path('./dataset')
root_ds2 = Path('./datasets2')
root_ds3 = Path('./dataset3/LP_detection')  # use LP_detection split in dataset3

assert root_main.exists(), "./dataset must exist (target)"
print('Target dataset:', root_main.resolve())

# Ensure subfolders
for p in [root_main/'images'/ 'train', root_main/'images'/'val', root_main/'labels'/'train', root_main/'labels'/'val']:
    p.mkdir(parents=True, exist_ok=True)

# Helper to copy tree contents (images + labels) preserving names
IMG_EXTS = {'.jpg','.jpeg','.png','.bmp','.tif','.tiff','.webp'}

def copy_split(src_images: Path, src_labels: Path, dst_images: Path, dst_labels: Path):
    if src_images and src_images.exists():
        files = [p for p in src_images.rglob('*') if p.suffix.lower() in IMG_EXTS]
        print(f'Copying {len(files)} images from', src_images)
        for fp in files:
            rel = fp.name
            shutil.copy2(fp, dst_images/rel)
    if src_labels and src_labels.exists():
        files = list(src_labels.rglob('*.txt'))
        print(f'Copying {len(files)} labels from', src_labels)
        for fp in files:
            rel = fp.name
            shutil.copy2(fp, dst_labels/rel)

# Merge from datasets2 (assumes YOLO layout)
if root_ds2.exists():
    copy_split(root_ds2/'images'/'train', root_ds2/'labels'/'train', root_main/'images'/'train', root_main/'labels'/'train')
    copy_split(root_ds2/'images'/'val',   root_ds2/'labels'/'val',   root_main/'images'/'val',   root_main/'labels'/'val')
else:
    print('datasets2 not found, skip')

# Merge from dataset3/LP_detection (assumes YOLO layout)
if root_ds3.exists():
    copy_split(root_ds3/'images'/'train', root_ds3/'labels'/'train', root_main/'images'/'train', root_main/'labels'/'train')
    copy_split(root_ds3/'images'/'val',   root_ds3/'labels'/'val',   root_main/'images'/'val',   root_main/'labels'/'val')
else:
    print('dataset3/LP_detection not found, skip')

# Clear caches so Ultralytics reindexes
for cache_name in ['train.cache', 'val.cache']:
    cp = root_main / 'labels' / cache_name
    if cp.exists():
        try:
            os.remove(cp)
            print('Removed cache:', cp)
        except Exception as e:
            print('Failed to remove cache', cp, e)

print('Merge done. Now re-run the data.yaml build cell above to re-assert paths if needed.')


In [ ]:
# 3) Train
from ultralytics import YOLO
from pathlib import Path
import torch, os, logging

# Show full per-epoch logs (progress bar + metrics)
os.environ['TQDM_DISABLE'] = '0'  # enable tqdm progress bars
os.environ.setdefault('ULTRALYTICS_HUB', '0')
os.environ.setdefault('WANDB_DISABLED', 'true')
try:
    from ultralytics.utils import LOGGER
    LOGGER.setLevel(logging.INFO)  # show epoch summaries and metrics
except Exception:
    pass

# Use local data.yaml and local best.pt only
yaml_path = Path('./dataset/data.yaml')
assert yaml_path.exists(), "dataset/data.yaml not found. Run the dataset cell first."

ckpt = str(Path('best.pt').resolve())
assert os.path.exists(ckpt), "best.pt not found at project root. Place your checkpoint at ./best.pt"

print(f'Training from: {ckpt}')
model = YOLO(ckpt)

# Training args with visible epoch metrics
train_args = dict(
    data=str(yaml_path),
    imgsz=640,        # bigger image size helps pose accuracy
    epochs=200,       # train up to 200 epochs
    patience=30,      # early stop if no val improvement for 30 epochs
    batch=4,          # adjust if you hit OOM
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=4,        # avoid multiprocessing issues in notebooks/Python 3.13
    project='runs',
    name='pose_plate',
    exist_ok=True,
    pretrained=False,  # avoid triggering any default weight downloads
    cache='ram',
    plots=True,       # save results.png and curves
    save_period=0,    # checkpoint every N epochs (0=only best/last)
    seed=42,
    amp=False,        # disable AMP to avoid online AMP check (no yolo11n.pt)
    verbose=True,     # print full per-epoch metrics
)

results = model.train(**train_args)
print('Training done. Best:', results)

In [ ]:
# 4) Validate & quick inference
from glob import glob
import os, yaml
import cv2, numpy as np

# Strict offline mode: allow local assets but block any remote downloads
os.environ.setdefault('ULTRALYTICS_HUB', '0')
os.environ.setdefault('WANDB_DISABLED', 'true')
try:
    from pathlib import Path as _P
    from ultralytics.utils import downloads as _ud
    from ultralytics.utils import checks as _uc
    _ud.is_online = lambda: False  # type: ignore

    def _local_only(asset, *args, **kwargs):
        # Return the local path if it exists; otherwise block
        try:
            p = _P(asset)
        except TypeError:
            try:
                p = _P(asset[0])
            except Exception:
                p = None
        if p is not None and p.exists():
            return str(p)
        raise RuntimeError(f"Ultralytics offline: asset not found locally: {asset}")

    for _name in ("safe_download", "attempt_download", "attempt_download_asset", "get_github_assets"):
        if hasattr(_ud, _name):
            setattr(_ud, _name, _local_only)
    if hasattr(_uc, "check_requirements"):
        _uc.check_requirements = lambda *a, **k: None  # type: ignore
except Exception:
    pass

from ultralytics import YOLO
from pathlib import Path

# Force local checkpoint only (absolute path)
ckpt = str(Path('best.pt').resolve())
assert os.path.exists(ckpt), "Local checkpoint './best.pt' not found. Place your trained weights as best.pt at project root."
print('Loading weights:', ckpt)
model = YOLO(ckpt)
print('Model task:', getattr(model, 'task', None))

# read data.yaml to find val images path
with open('./dataset/data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)
val_path = data_cfg.get('val')
base_path = data_cfg.get('path', '.')
val_dir = os.path.join(base_path, val_path) if val_path else './dataset/val/images'

# pick a few images
val_imgs = []
for pat in ['*.jpg', '*.jpeg', '*.png', '*.*']:
    val_imgs = glob(os.path.join(val_dir, pat))
    if val_imgs:
        break
val_imgs = val_imgs[:6]
print('Previewing', len(val_imgs), 'images from', val_dir)


def _to_numpy(x):
    try:
        import torch
        if isinstance(x, np.ndarray):
            return x
        if isinstance(x, torch.Tensor):
            return x.detach().cpu().numpy()
    except Exception:
        pass
    return np.asarray(x)


def _order_points_four(pts: np.ndarray) -> np.ndarray:
    pts = np.asarray(pts, dtype='float32')
    if pts.shape[0] != 4:
        return pts
    s = pts.sum(axis=1)
    diff = pts[:, 1] - pts[:, 0]
    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype='float32')


for imgp in val_imgs:
    im = cv2.imread(imgp)
    res = model.predict(source=im, imgsz=640, conf=0.5, iou=0.6, classes=[0], verbose=False, max_det=50)[0]

    # Draw boxes
    if res.boxes is not None and len(res.boxes) > 0:
        for b in res.boxes:
            x1,y1,x2,y2 = b.xyxy[0].int().cpu().tolist()
            cv2.rectangle(im, (x1,y1), (x2,y2), (0,255,0), 2)

    # Draw keypoints with CPU conversion and polygon
    if getattr(res, 'keypoints', None) is not None and res.keypoints.xy is not None:
        kxy = _to_numpy(res.keypoints.xy)
        kcf = _to_numpy(getattr(res.keypoints, 'conf', None)) if getattr(res.keypoints, 'conf', None) is not None else None
        for i in range(kxy.shape[0]):
            pts = kxy[i]
            if pts is None or pts.shape[0] < 4:
                continue
            mask = np.ones((pts.shape[0],), dtype=bool)
            if kcf is not None:
                mask = kcf[i] >= 0.1
            pts_vis = pts[mask]
            if pts_vis.shape[0] >= 4:
                pts4 = pts_vis[:4]
                ordered = _order_points_four(pts4)
                poly = ordered.reshape((-1,1,2)).astype(int)
                cv2.polylines(im, [poly], isClosed=True, color=(0,0,255), thickness=2)
                for px, py in ordered:
                    cv2.circle(im, (int(px), int(py)), 4, (0,255,255), -1)
            else:
                for px, py in pts[:4]:
                    cv2.circle(im, (int(px), int(py)), 4, (0,255,255), -1)

    cv2.imshow('preview', im)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
# 3.a) Quick sanity checks for validation split (to explain empty metrics warnings)
from pathlib import Path
from collections import Counter
import yaml, glob, os

# Load dataset config
with open('./dataset/data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)

base = Path(data_cfg.get('path', './dataset')).resolve()
val_rel = data_cfg.get('val')
val_dir = (base / val_rel).resolve() if val_rel else (base / 'images' / 'val').resolve()

# Try to locate labels/val
label_dirs = [base / 'labels' / 'val', base / 'val' / 'labels']
labels_dir = next((p for p in label_dirs if p.exists()), None)

# Collect images
IMG_EXTS = ('*.jpg','*.jpeg','*.png','*.bmp','*.webp')
val_imgs = []
for ext in IMG_EXTS:
    val_imgs += glob.glob(str(val_dir / ext))
val_imgs = sorted(set(val_imgs))

# Collect labels
label_files = []
if labels_dir and labels_dir.exists():
    label_files = sorted(glob.glob(str(labels_dir / '*.txt')))

# Pairing check (image <-> label)
img_stems = {Path(p).stem for p in val_imgs}
lab_stems = {Path(p).stem for p in label_files}
missing_labels = sorted(img_stems - lab_stems)[:20]
missing_images = sorted(lab_stems - img_stems)[:20]

# Label content check (YOLO-pose expects: cls cx cy w h + 3*K)
valid_pose_lines = 0
k_counter = Counter()
empty_label_files = 0
kpt_only_lines = 0

for fp in label_files[:2000]:
    try:
        with open(fp, 'r') as f:
            lines = [ln.strip() for ln in f if ln.strip()]
        if not lines:
            empty_label_files += 1
            continue
        for ln in lines[:4]:
            parts = ln.split()
            if len(parts) >= 8 and (len(parts) - 5) % 3 == 0:
                valid_pose_lines += 1
                Kcand = (len(parts) - 5) // 3
                k_counter[Kcand] += 1
            elif len(parts) >= 3 and (len(parts) - 1) % 2 == 0:
                # likely keypoints-only (no bbox) -> will yield empty detection metrics
                kpt_only_lines += 1
    except Exception:
        continue

print('Validation images:', len(val_imgs))
print('Validation label files:', len(label_files))
print('Empty label files:', empty_label_files)
print('Sample missing label stems (first 20):', missing_labels)
print('Sample labels without matching images (first 20):', missing_images)
print('Valid YOLO-pose lines seen:', valid_pose_lines, '| kpt-only lines (no bbox):', kpt_only_lines)
print('Inferred K candidates (from labels):', dict(k_counter))

if len(val_imgs) == 0:
    print('[Hint] No validation images. Metrics arrays will be empty -> warnings. Ensure data.yaml val points to images/val and files exist.')
elif len(label_files) == 0:
    print('[Hint] No validation labels found. Place .txt under labels/val with YOLO-pose format (cls cx cy w h + 3*K).')
elif valid_pose_lines == 0 and kpt_only_lines > 0:
    print('[Hint] Labels look keypoints-only (no bbox). Ultralytics pose expects bbox + keypoints; add boxes or convert labels.')
elif missing_labels:
    print('[Hint] Some images have no corresponding label .txt. Create labels or remove those images from val set.')
else:
    print('Sanity checks look OK. Early-epoch warnings can still appear if the model predicts nothing yet; they usually disappear later.')

In [ ]:
# 3.b) Auto-fix val labels: convert keypoints-only -> bbox+keypoints, create empty labels for unpaired images
from pathlib import Path
import yaml, os, glob, shutil

# Settings
DO_CREATE_EMPTY_LABELS = True   # create empty .txt for images without labels
VIS_DEFAULT = 2                 # default visibility if missing in kpt-only labels (0=not labeled, 1=occluded, 2=visible)
FLOAT_FMT = '{:.6f}'.format

# Load dataset config and resolve dirs
with open('./dataset/data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)
base = Path(data_cfg.get('path', './dataset')).resolve()
val_rel = data_cfg.get('val')
val_dir = (base / val_rel).resolve() if val_rel else (base / 'images' / 'val').resolve()
label_dirs = [base / 'labels' / 'val', base / 'val' / 'labels']
labels_dir = next((p for p in label_dirs if p.exists()), None)
assert val_dir.exists(), f"Val images dir not found: {val_dir}"
assert labels_dir is not None, "Could not find labels/val folder. Create it under dataset/labels/val or dataset/val/labels."

# Backup dir
backup_dir = base / 'labels_backup_before_pose_fix' / 'val'
backup_dir.mkdir(parents=True, exist_ok=True)

IMG_EXTS = ('.jpg','.jpeg','.png','.bmp','.webp')

# Build index of val images by stem
img_map = {}
for p in val_dir.iterdir():
    if p.suffix.lower() in IMG_EXTS:
        img_map[p.stem] = p

# List label files
label_files = sorted(glob.glob(str(labels_dir / '*.txt')))
lab_stems = {Path(p).stem for p in label_files}
img_stems = set(img_map.keys())

# Create empty labels for images without labels (optional)
created_empty = 0
if DO_CREATE_EMPTY_LABELS:
    for stem in sorted(img_stems - lab_stems):
        outp = labels_dir / f"{stem}.txt"
        if not outp.exists():
            outp.write_text('')
            created_empty += 1

# Helper to detect if coordinates are normalized [0,1]
def _coords_look_normalized(vals):
    # if any coord > 1.0 or < 0.0, likely absolute pixels
    return all(0.0 <= v <= 1.0 for v in vals)

# Convert kpt-only lines to bbox+keypoints
converted_files = 0
converted_lines = 0
skipped_lines_noimg = 0

for lfp in label_files:
    stem = Path(lfp).stem
    with open(lfp, 'r') as f:
        lines = [ln.strip() for ln in f]
    if not any(ln for ln in lines):
        continue

    changed = False
    new_lines = []
    for ln in lines:
        if not ln.strip():
            new_lines.append(ln)
            continue
        parts = ln.split()
        # full pose format: cls cx cy w h + 3*K
        if len(parts) >= 8 and (len(parts) - 5) % 3 == 0:
            new_lines.append(ln)
            continue
        # keypoints-only: cls x1 y1 x2 y2 ...
        if len(parts) >= 3 and (len(parts) - 1) % 2 == 0:
            cls_id = parts[0]
            coords = list(map(float, parts[1:]))
            xs = coords[0::2]
            ys = coords[1::2]
            normalized = _coords_look_normalized(xs + ys)
            # If not normalized, try to normalize using image size
            if not normalized:
                im = img_map.get(stem)
                if im is None:
                    # cannot normalize without image
                    new_lines.append(ln)
                    skipped_lines_noimg += 1
                    continue
                try:
                    import cv2
                    im0 = cv2.imread(str(im))
                    if im0 is None:
                        new_lines.append(ln)
                        skipped_lines_noimg += 1
                        continue
                    h, w = im0.shape[:2]
                    xs = [x / w for x in xs]
                    ys = [y / h for y in ys]
                    normalized = True
                except Exception:
                    new_lines.append(ln)
                    skipped_lines_noimg += 1
                    continue
            # compute bbox from normalized kpts
            min_x, max_x = max(0.0, min(xs)), min(1.0, max(xs))
            min_y, max_y = max(0.0, min(ys)), min(1.0, max(ys))
            bw = max(1e-6, max_x - min_x)
            bh = max(1e-6, max_y - min_y)
            cx = (min_x + max_x) / 2.0
            cy = (min_y + max_y) / 2.0
            # build v flags (default visible)
            K = len(xs)
            kpts_fmt = []
            for x, y in zip(xs, ys):
                kpts_fmt.extend([FLOAT_FMT(x), FLOAT_FMT(y), str(VIS_DEFAULT)])
            new_ln = ' '.join([cls_id, FLOAT_FMT(cx), FLOAT_FMT(cy), FLOAT_FMT(bw), FLOAT_FMT(bh)] + kpts_fmt)
            new_lines.append(new_ln)
            converted_lines += 1
            changed = True
        else:
            new_lines.append(ln)

    if changed:
        # backup once per file
        bk = backup_dir / f"{Path(lfp).name}"
        if not bk.exists():
            shutil.copy2(lfp, bk)
        with open(lfp, 'w') as f:
            f.write('\n'.join(new_lines).strip() + '\n')
        converted_files += 1

print('Auto-fix summary:')
print(' - Created empty labels:', created_empty)
print(' - Converted files:', converted_files)
print(' - Converted lines:', converted_lines)
print(' - Skipped lines (no image to normalize):', skipped_lines_noimg)
print('Backup of originals:', backup_dir)
print('Tip: Re-run the sanity check cell (3.a) to verify improvements, then retrain.')

In [ ]:
# 3.c) Clear Ultralytics label caches (so updated labels take effect)
from pathlib import Path
import os

root = Path('./dataset')
for cache_name in ['train.cache', 'val.cache']:
    cp = root / 'labels' / cache_name
    if cp.exists():
        try:
            os.remove(cp)
            print('Removed cache:', cp)
        except Exception as e:
            print('Failed to remove cache', cp, e)
    else:
        print('Cache not found (ok):', cp)
print('Caches cleared. Re-run sanity check (3.a) if desired, then train (3).')

In [ ]:
# 3.d) Inspect label column widths (train & val) and expected K
from pathlib import Path
import yaml, glob, os
from collections import Counter, defaultdict

# Load dataset config
with open('./dataset/data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)
base = Path(data_cfg.get('path', './dataset')).resolve()

# Resolve split dirs
split_img_dirs = {
    'train': (base / (data_cfg.get('train') or 'images/train')).resolve(),
    'val':   (base / (data_cfg.get('val')   or 'images/val')).resolve(),
}
split_lbl_dirs = {
    'train': next((p for p in [base/'labels'/'train', base/'train'/'labels'] if p.exists()), None),
    'val':   next((p for p in [base/'labels'/'val',   base/'val'/'labels']   if p.exists()), None),
}

print('Label directories:', {k: str(v) if v else None for k,v in split_lbl_dirs.items()})

# Scan labels to compute per-line width and infer K per split
summary = {}
offenders = defaultdict(list)

for split, ldir in split_lbl_dirs.items():
    widths = Counter()
    k_counts = Counter()
    det_only = 0
    kpt_only = 0
    invalid = 0
    files = []
    if ldir and ldir.exists():
        files = sorted(glob.glob(str(ldir / '*.txt')))
    for lfp in files:
        try:
            with open(lfp, 'r') as f:
                lines = [ln.strip() for ln in f if ln.strip()]
        except Exception:
            continue
        for ln in lines:
            parts = ln.split()
            n = len(parts)
            if n == 5:
                det_only += 1
                widths[n] += 1
                offenders[(split, 'detection_only')].append(lfp)
            elif n >= 8 and (n - 5) % 3 == 0:
                K = (n - 5) // 3
                k_counts[K] += 1
                widths[n] += 1
            elif n >= 3 and (n - 1) % 2 == 0:
                kpt_only += 1
                widths[n] += 1
                offenders[(split, 'kpt_only_no_bbox')].append(lfp)
            else:
                invalid += 1
                widths[n] += 1
                offenders[(split, 'invalid_width')].append(lfp)
    summary[split] = dict(
        files=len(files),
        widths=dict(widths),
        inferred_K=dict(k_counts),
        detection_only=det_only,
        kpt_only_no_bbox=kpt_only,
        invalid=invalid,
    )

print('Summary (by split):')
for split, info in summary.items():
    print(f'  {split}:')
    for k, v in info.items():
        print(f'    {k}: {v}')

# If a dominant K exists, compute expected columns = 5 + 3*K
all_k = Counter()
for s in summary.values():
    for k, c in s['inferred_K'].items():
        all_k[k] += c
expected_K = max(all_k, key=all_k.get) if all_k else None
expected_cols = 5 + 3*expected_K if expected_K is not None else None
print('Inferred dominant K across splits:', expected_K, '| expected columns per line:', expected_cols)

# Show a few offender files per type
for (split, typ), lst in offenders.items():
    if lst:
        print(f'Offenders [{split}::{typ}] (up to 10):')
        for p in sorted(set(lst))[:10]:
            print('  -', os.path.basename(p))

print('Tip: For pose, each line must be: cls cx cy w h + 3*K (x y v) with K keypoints, normalized [0,1].')

In [ ]:
# 3.e) Normalize labels to K=4 pose format across train & val
from pathlib import Path
import yaml, os, glob, shutil

# Settings
VIS_DEFAULT = 2                 # visibility for generated keypoints
FLOAT_FMT = '{:.6f}'.format

# Load dataset config and resolve dirs
with open('./dataset/data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)
base = Path(data_cfg.get('path', './dataset')).resolve()

# Resolve split folders
split_to_img = {
    'train': (base / (data_cfg.get('train') or 'images/train')).resolve(),
    'val':   (base / (data_cfg.get('val')   or 'images/val')).resolve(),
}
split_to_lbl = {
    'train': next((p for p in [base/'labels'/'train', base/'train'/'labels'] if p.exists()), None),
    'val':   next((p for p in [base/'labels'/'val',   base/'val'/'labels']   if p.exists()), None),
}

# Backups
backup_root = base / 'labels_backup_before_pose_fix'

IMG_EXTS = {'.jpg','.jpeg','.png','.bmp','.webp'}

def bbox_corners(cx, cy, w, h):
    x1 = max(0.0, cx - w/2)
    y1 = max(0.0, cy - h/2)
    x2 = min(1.0, cx + w/2)
    y2 = min(1.0, cy + h/2)
    # Order: tl, tr, br, bl
    return [(x1,y1),(x2,y1),(x2,y2),(x1,y2)]

def coords_look_normalized(vals):
    return all(0.0 <= v <= 1.0 for v in vals)

# Build image maps for normalization when needed
split_img_map = {}
for split, idir in split_to_img.items():
    imap = {}
    if idir and idir.exists():
        for p in idir.iterdir():
            if p.suffix.lower() in IMG_EXTS:
                imap[p.stem] = p
    split_img_map[split] = imap

stats = {
    'train_det_to_pose': 0,
    'train_kpt_only_to_pose': 0,
    'val_k2_to_k4': 0,
    'files_touched': 0,
}

for split in ('train','val'):
    ldir = split_to_lbl[split]
    if not ldir or not ldir.exists():
        continue
    (backup_root/split).mkdir(parents=True, exist_ok=True)
    files = sorted(glob.glob(str(ldir / '*.txt')))
    for lfp in files:
        stem = Path(lfp).stem
        with open(lfp, 'r') as f:
            lines = [ln.rstrip('\n') for ln in f]
        new_lines = []
        changed = False
        for ln in lines:
            if not ln.strip():
                new_lines.append(ln)
                continue
            parts = ln.split()
            n = len(parts)
            # Detection-only -> add 4 bbox-corner keypoints
            if n == 5:
                cls_id, cx, cy, w, h = parts
                cx = float(cx); cy = float(cy); w = float(w); h = float(h)
                kpts = []
                for x,y in bbox_corners(cx, cy, w, h):
                    kpts.extend([FLOAT_FMT(x), FLOAT_FMT(y), str(VIS_DEFAULT)])
                new_lines.append(' '.join([cls_id, FLOAT_FMT(cx), FLOAT_FMT(cy), FLOAT_FMT(w), FLOAT_FMT(h)] + kpts))
                changed = True
                if split == 'train':
                    stats['train_det_to_pose'] += 1
                continue
            # Pose with K=2 -> replace keypoints with bbox corners to make K=4
            if n == 11 and (n - 5) % 3 == 0:
                cls_id, cx, cy, w, h = parts[:5]
                cx = float(cx); cy = float(cy); w = float(w); h = float(h)
                kpts = []
                for x,y in bbox_corners(cx, cy, w, h):
                    kpts.extend([FLOAT_FMT(x), FLOAT_FMT(y), str(VIS_DEFAULT)])
                new_lines.append(' '.join([cls_id, FLOAT_FMT(cx), FLOAT_FMT(cy), FLOAT_FMT(w), FLOAT_FMT(h)] + kpts))
                changed = True
                if split == 'val':
                    stats['val_k2_to_k4'] += 1
                continue
            # Keypoints-only -> build bbox and add vis flags
            if n >= 3 and (n - 1) % 2 == 0:
                cls_id = parts[0]
                coords = list(map(float, parts[1:]))
                xs = coords[0::2]
                ys = coords[1::2]
                normalized = coords_look_normalized(xs + ys)
                if not normalized:
                    im = split_img_map[split].get(stem)
                    if im is not None:
                        try:
                            import cv2
                            im0 = cv2.imread(str(im))
                            h0, w0 = im0.shape[:2]
                            xs = [x / w0 for x in xs]
                            ys = [y / h0 for y in ys]
                            normalized = True
                        except Exception:
                            pass
                if normalized:
                    min_x, max_x = max(0.0, min(xs)), min(1.0, max(xs))
                    min_y, max_y = max(0.0, min(ys)), min(1.0, max(ys))
                    bw = max(1e-6, max_x - min_x)
                    bh = max(1e-6, max_y - min_y)
                    cx = (min_x + max_x) / 2.0
                    cy = (min_y + max_y) / 2.0
                    # Generate 4 bbox-corner keypoints regardless of original K to keep consistent K=4
                    kpts = []
                    for x,y in bbox_corners(cx, cy, bw, bh):
                        kpts.extend([FLOAT_FMT(x), FLOAT_FMT(y), str(VIS_DEFAULT)])
                    new_lines.append(' '.join([cls_id, FLOAT_FMT(cx), FLOAT_FMT(cy), FLOAT_FMT(bw), FLOAT_FMT(bh)] + kpts))
                    changed = True
                    if split == 'train':
                        stats['train_kpt_only_to_pose'] += 1
                    continue
                else:
                    # cannot normalize; keep original line
                    new_lines.append(ln)
                    continue
            # Already pose with K>=4 or other -> keep as-is
            new_lines.append(ln)
        if changed:
            bk = backup_root / split / f'{Path(lfp).name}'
            if not bk.exists():
                shutil.copy2(lfp, bk)
            with open(lfp, 'w') as f:
                f.write('\n'.join(new_lines).strip() + '\n')
            stats['files_touched'] += 1

print('Normalize summary:')
for k,v in stats.items():
    print(f' - {k}: {v}')
print('Backups saved under:', backup_root)
print('Tip: Run cell 3.c to clear caches, then re-run 3.d to verify widths are now 17 across splits, and retrain (3).')

In [ ]:
# 3.f) Fix invalid class IDs (e.g., class 1 when dataset has only class 0)
from pathlib import Path
import yaml, glob, shutil
from collections import Counter

# Load dataset config
with open('./dataset/data.yaml', 'r') as f:
    data_cfg = yaml.safe_load(f)

base = Path(data_cfg.get('path', './dataset')).resolve()
# Determine number of classes from names
names = data_cfg.get('names', ['plate'])
if isinstance(names, dict):
    # names: {0: 'plate'}
    class_count = len(names)
elif isinstance(names, (list, tuple)):
    class_count = len(names)
else:
    class_count = 1

valid_min, valid_max = 0, max(0, class_count - 1)

# Resolve split folders
split_to_lbl = {
    'train': next((p for p in [base/'labels'/'train', base/'train'/'labels'] if p.exists()), None),
    'val':   next((p for p in [base/'labels'/'val',   base/'val'/'labels']   if p.exists()), None),
}

backup_root = base / 'labels_backup_before_pose_fix' / 'class_fix'
backup_root.mkdir(parents=True, exist_ok=True)

stats_split = {}

def parse_cls(tok: str):
    try:
        # handle '0', '0.0', etc.
        return int(float(tok))
    except Exception:
        return None

for split, ldir in split_to_lbl.items():
    if not ldir or not ldir.exists():
        continue
    files = sorted(glob.glob(str(ldir / '*.txt')))
    changed_files = 0
    total_lines = 0
    changed_lines = 0
    invalid_tokens = 0
    cls_hist_before = Counter()
    cls_hist_after = Counter()

    (backup_root / split).mkdir(parents=True, exist_ok=True)

    for lfp in files:
        with open(lfp, 'r') as f:
            lines = [ln.rstrip('\n') for ln in f]
        new_lines = []
        file_changed = False
        for ln in lines:
            if not ln.strip():
                new_lines.append(ln)
                continue
            parts = ln.split()
            if len(parts) < 5:
                # not a proper YOLO line; keep as-is
                new_lines.append(ln)
                continue
            cid = parse_cls(parts[0])
            total_lines += 1
            if cid is None:
                invalid_tokens += 1
                cid = 0
            cls_hist_before[cid] += 1
            new_cid = min(max(cid, valid_min), valid_max)
            if new_cid != cid:
                parts[0] = str(new_cid)
                file_changed = True
                changed_lines += 1
            new_lines.append(' '.join(parts))
            cls_hist_after[new_cid] += 1
        if file_changed:
            bk = backup_root / split / Path(lfp).name
            if not bk.exists():
                shutil.copy2(lfp, bk)
            with open(lfp, 'w') as f:
                f.write('\n'.join(new_lines).strip() + '\n')
            changed_files += 1

    stats_split[split] = {
        'files': len(files),
        'changed_files': changed_files,
        'total_lines': total_lines,
        'changed_lines': changed_lines,
        'invalid_tokens': invalid_tokens,
        'cls_hist_before': dict(cls_hist_before),
        'cls_hist_after': dict(cls_hist_after),
    }

print('Class-fix summary (valid IDs =', f'{valid_min}-{valid_max}', '):')
for split, st in stats_split.items():
    print(f'  {split}:')
    for k, v in st.items():
        if isinstance(v, dict):
            print(f'    {k}: {dict(sorted(v.items()))}')
        else:
            print(f'    {k}: {v}')
print('Backups saved under:', backup_root)
print('Tip: Run cell 3.c to clear caches, then retrain (3).')